# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.7.0'

## Part 1 - Data Preprocessing

##### ข้อมูลเพิ่มเติม: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

### Preprocessing the Training set

In [3]:
# ImageDataGenerator จะทำการซุมเข้า ซุมออก หมุนภาพ เพื่อไม่ให้ model overfitting
train_datagen = ImageDataGenerator(
        #Feature Scaling / pixel values มี 255 ถ้าหารแต่ละ pixel ด้วย 255 ก็จะได้ค่าระหว่าง 1 กับ 0
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
train_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        # ขนาดของภาพ
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
# filters : จำนวน Convolutional layer
# kernel_size : ขนาดของ Feature Detector / 3 คือ Matrix 3X3
# input_shape : ขนาดของภาพ (64X64), ภาพสีRGB (3), ภาพขาวดำ (1)
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [64, 64, 3]))

### Step 2 - Pooling

In [7]:
#pool_size : ขนาดของ Max pooling / 2* คือ Matrix 2X2
#strides : การขยับของ max pooling ไป pixel ต่อไป / 2* คือ ขยับไปทีละ 2 pixel
# * recommended input
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### Adding a second convolutional layer

In [8]:
# layer ที่ 2 ไม่ต้องใส่ input_shape
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))

### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units = 1, activation ='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
cnn.fit(x = train_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 53s 194ms/step - loss: 0.6707 - accuracy: 0.5875 - val_loss: 0.6744 - val_accuracy: 0.6105
Epoch 2/25
250/250 [==============================] - 18s 70ms/step - loss: 0.6079 - accuracy: 0.6756 - val_loss: 0.5669 - val_accuracy: 0.7115
Epoch 3/25
250/250 [==============================] - 18s 72ms/step - loss: 0.5618 - accuracy: 0.7138 - val_loss: 0.5194 - val_accuracy: 0.7470
Epoch 4/25
250/250 [==============================] - 18s 71ms/step - loss: 0.5218 - accuracy: 0.7399 - val_loss: 0.4986 - val_accuracy: 0.7605
Epoch 5/25
250/250 [==============================] - 18s 71ms/step - loss: 0.4966 - accuracy: 0.7549 - val_loss: 0.4883 - val_accuracy: 0.7680
Epoch 6/25
250/250 [==============================] - 18s 71ms/step - loss: 0.4860 - accuracy: 0.7665 - val_loss: 0.4955 - val_accuracy: 0.7700
Epoch 7/25
250/250 [==============================] - 18s 71ms/step - loss: 0.4732 - accuracy: 0.7686 - val_loss: 0.4983 - val_accuracy

## Part 4 - Making a single prediction

In [27]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_3.jpg', target_size = (64, 64))
# change image to array
test_image = image.img_to_array(test_image)
# เพิ่ม dimension ให้ภาพ เพื่อให้ format ตรงกับ train set ที่มี batch size (axis = 0: row)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
#encord 0 and 1 to cat and dog
train_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

In [28]:
print(prediction)

dog
